In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
glft_dir = "../../results/daily/glft"
data_glft = np.load(f"{glft_dir}/day_03/execution_times_nano.npy") / 1e6

In [ ]:
glft_dir = "../../results/daily/glft"
rf_dir = "../../results/daily/rf-dynamic-train-and-norm-tight"

data_glft = np.load(f"{glft_dir}/day_02/execution_times_nano.npy")
data_rf = np.load(f"{rf_dir}/day_02/execution_times_seconds.npy")


for day in range(3, 32):
    glft_file = f"{glft_dir}/day_{day:02d}/execution_times_nano.npy"
    rf_file = f"{rf_dir}/day_{day:02d}/execution_times_seconds.npy"
    
    glft_df = np.load(glft_file) / 1e6
    rf_df = np.load(rf_file) * 1000
    
    last_non_zero_idx_rf = np.nonzero(rf_df)[0][-1]
    last_non_zero_idx_glft = np.nonzero(glft_df)[0][-1]
    
    data_rf_ms = rf_df[:last_non_zero_idx_rf + 1]
    data_glft_ms = glft_df[:last_non_zero_idx_glft + 1]
    
    
    data_glft = np.concatenate((data_glft, data_glft_ms))
    data_rf = np.concatenate((data_rf, data_rf_ms))

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, np.inf]
labels = [
    '<1 ms', '1-2 ms', '2-3 ms', '3-4 ms', '4-5 ms', '5-6 ms', '6> ms']

# for glft
for i in range(len(bins)-1):
    count = np.sum((data_glft_ms >= bins[i]) & (data_glft_ms < bins[i+1]))
    percentage = count / len(data_glft_ms) * 100
    print(f'GLFT: {labels[i]}: {count} ({percentage:.2f}%)')
    
# for rf
for i in range(len(bins)-1):
    count = np.sum((data_rf_ms >= bins[i]) & (data_rf_ms < bins[i+1]))
    percentage = count / len(data_rf_ms) * 100
    print(f'RF: {labels[i]}: {count} ({percentage:.2f}%)')

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, np.inf]
labels = [f'{bins[i]}-{bins[i+1]} ms' if np.isfinite(bins[i+1])
          else f'>{bins[i]} ms'
          for i in range(len(bins)-1)]

counts, _ = np.histogram(data_rf, bins=bins)
percentages = counts / counts.sum() * 100

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(labels, counts, alpha=0.8)

for bar, pct in zip(bars, percentages):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01*counts.max(),
            f'{pct:.1f}%', ha='center', va='bottom', fontsize=9)

ax.set_xlabel('Duration (milliseconds)')
ax.set_ylabel('Tick Counts')
ax.set_title('GLFT+RFC Tick Execution Times')
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.grid(axis='y', linestyle='--', linewidth=0.5, alpha=0.6)
plt.tight_layout()
plt.savefig("../../plots/execution_times/glft_rfc_execution_times.png")
plt.show()


In [ ]:
bins = [0,0.01, 0.02, 0.03, 0.04, 0.05, np.inf]
labels = ['<0.01 ms', '0.01-0.02 ms', '0.02-0.03 ms', '0.03-0.04 ms', '0.04-0.05 ms', '>0.05 ms']

counts = np.histogram(data_glft, bins=bins)[0]
count = np.sum(counts)
percentages = (counts / count) * 100

plt.figure(figsize=(10, 6))
bars = plt.bar(labels, counts, alpha=0.7, label='GLFT+RF')

for bar, pct in zip(bars, percentages):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 1, f'{pct:.1f}%', 
             ha='center', va='bottom', fontsize=10)

plt.xlabel('Duration (milliseconds)')
plt.ylabel('Tick Counts')
plt.title('Base GLFT Execution Times')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig("../../plots/execution_times/glft_execution_times.png")
plt.show()

In [ ]:
tree_times = []

for day in range(2, 32):
    file_path = f"../../results/daily/rf-tight-spread-constant-train-dynamic-norm/day_{day:02d}/tree_train_times_seconds.npy"
    train_times = np.load(file_path)
    print(train_times)

labels = [f"Jan {i+2}" for i in range(len(tree_times))]
plt.figure(figsize=(12, 6))
bars = plt.bar(labels, tree_times / 60, alpha=0.7)

    
plt.xlabel('Date (January 2025)')
plt.ylabel('Training Time (minutes)')
plt.title('RandomForestClassifier Training Times')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("../../plots/execution_times/rf_tree_train_times.png")
plt.show()